<a href="https://colab.research.google.com/github/csrviswa/Coursera_Capstone/blob/master/Neighborhoods_Analysis_Capstone_Project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import pip    
from bs4 import BeautifulSoup
import requests
import csv
import pandas as pd
!pip install folium==0.5.0
import folium
import numpy as np
from sklearn.cluster import KMeans
import matplotlib.cm as cm
import matplotlib.colors as colors


## To explore Neighborhoods in Toronto

In [0]:
source = requests.get('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M').text
soup = BeautifulSoup(source, 'lxml')
table = soup.find('table', class_='wikitable')
csv_file = open('postal_codes_of_Canada.csv', 'w')
csv_writer = csv.writer(csv_file)
coloumn = []
row = [];
for th in table.find_all('th'):
    coloumn.append(th.text.replace('\n', ''))
csv_writer.writerow(coloumn)
for td in table.find_all('td'):
    if(td.text.find('\n') != -1) :
       row.append(td.text.replace('\n', '')) 
       csv_writer.writerow(row)
       row.clear()
    else :
       row.append(td.text)
      
csv_file.close()    
data = pd.read_csv("postal_codes_of_Canada.csv") 
data.drop(data[data.Borough == 'Not assigned'].index, inplace=True)
data.reset_index(drop=True, inplace=True)
data = data.groupby(['Postcode','Borough'], sort=False).Neighbourhood.apply(', '.join).reset_index(name='Neighbourhood')
data.loc[data[data.Neighbourhood == 'Not assigned'].index, 'Neighbourhood'] = data.loc[data[data.Neighbourhood == 'Not assigned'].index, 'Borough']
print(data.shape)
data.head()


(103, 3)


,Postcode,Borough,Neighbourhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Harbourfront, Regent Park"
3,M6A,North York,"Lawrence Heights, Lawrence Manor"
4,M7A,Queen's Park,Queen's Park


In [0]:
pd.set_option("display.precision", 7)
geo_data = requests.get('https://cocl.us/Geospatial_data')
with open('geo_data.csv', 'wb') as f:
    f.write(geo_data.content)
geo_df = pd.read_csv("geo_data.csv") 
geo_df.rename(columns={'Postal Code': 'Postcode'}, inplace=True)
geo_df.head()
neighborhoods = pd.merge(data, geo_df, on=['Postcode'], how='inner')
neighborhoods.head()

,Postcode,Borough,Neighbourhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.7532586,-79.3296565
1,M4A,North York,Victoria Village,43.7258823,-79.3155716
2,M5A,Downtown Toronto,"Harbourfront, Regent Park",43.6542599,-79.3606359
3,M6A,North York,"Lawrence Heights, Lawrence Manor",43.7185180,-79.4647633
4,M7A,Queen's Park,Queen's Park,43.6623015,-79.3894938


## Filter neighborhoods in Toronto

In [0]:
toronto_neigh = neighborhoods[neighborhoods['Borough'].str.contains("Toronto")]
toronto_neigh.head()

,Postcode,Borough,Neighbourhood,Latitude,Longitude
2,M5A,Downtown Toronto,"Harbourfront, Regent Park",43.6542599,-79.3606359
9,M5B,Downtown Toronto,"Ryerson, Garden District",43.6571618,-79.3789371
15,M5C,Downtown Toronto,St. James Town,43.6514939,-79.3754179
19,M4E,East Toronto,The Beaches,43.6763574,-79.2930312
20,M5E,Downtown Toronto,Berczy Park,43.6447708,-79.3733064


In [0]:
CLIENT_ID = '4QXTMPMC5PZRWUAS0PGV4JE1ISINO5NMOSOQLOZ53QJ3YDF1' # your Foursquare ID
CLIENT_SECRET = 'U0MA3FFMZ2YNU5JJFL4NKEPBV25BMIPPQL5HABQFGOVKXKEX' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)
LIMIT = 100 # limit of number of venues returned by Foursquare API
radius = 500 # define radius

def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighbourhood', 
                  'Neighbourhood Latitude', 
                  'Neighbourhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

Your credentails:
CLIENT_ID: 4QXTMPMC5PZRWUAS0PGV4JE1ISINO5NMOSOQLOZ53QJ3YDF1
CLIENT_SECRET:U0MA3FFMZ2YNU5JJFL4NKEPBV25BMIPPQL5HABQFGOVKXKEX


##Analyze Each Neighborhood

In [0]:
toronto_venues = getNearbyVenues(names=toronto_neigh['Neighbourhood'],
                                   latitudes=toronto_neigh['Latitude'],
                                   longitudes=toronto_neigh['Longitude']
                                  )
toronto_venues.head()

,Neighbourhood,Neighbourhood Latitude,Neighbourhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,"Harbourfront, Regent Park",43.6542599,-79.3606359,Roselle Desserts,43.6534467,-79.3620167,Bakery
1,"Harbourfront, Regent Park",43.6542599,-79.3606359,Tandem Coffee,43.6535587,-79.3618095,Coffee Shop
2,"Harbourfront, Regent Park",43.6542599,-79.3606359,Toronto Cooper Koo Family Cherry St YMCA Centre,43.6531905,-79.3579470,Gym / Fitness Center
3,"Harbourfront, Regent Park",43.6542599,-79.3606359,Morning Glory Cafe,43.6539469,-79.3611488,Breakfast Spot
4,"Harbourfront, Regent Park",43.6542599,-79.3606359,Body Blitz Spa East,43.6547351,-79.3598743,Spa


In [0]:
toronto_venues1 = toronto_venues.loc[toronto_venues.iloc[:,6].str.contains(r'(Restaurant|Train|Airport|Bus|Breakfast|Coffee|Bakery|Fitness|Pizza|Shop|Store|Gym|Yoga|Café|Burrito|Rail|Joint|Market|Workshop|Pizza|Sandwich|Food|Steakhouse|Sandwich|Salad|Diner)') == False]
toronto_venues1.head()

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:1: UserWarning: This pattern has match groups. To actually get the groups, use str.extract.
  """Entry point for launching an IPython kernel.


,Neighbourhood,Neighbourhood Latitude,Neighbourhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
4,"Harbourfront, Regent Park",43.6542599,-79.3606359,Body Blitz Spa East,43.6547351,-79.3598743,Spa
7,"Harbourfront, Regent Park",43.6542599,-79.3606359,Dominion Pub and Kitchen,43.6569186,-79.3589668,Pub
8,"Harbourfront, Regent Park",43.6542599,-79.3606359,Corktown Common,43.6556178,-79.3562113,Park
9,"Harbourfront, Regent Park",43.6542599,-79.3606359,The Distillery Historic District,43.6502444,-79.3593228,Historic Site
19,"Harbourfront, Regent Park",43.6542599,-79.3606359,Young Centre for the Performing Arts,43.6508247,-79.3575932,Performing Arts Venue


In [0]:
map_toronto = folium.Map(location=[43.6532, -79.3832], zoom_start=12)

# add markers to map
for lat, lng, venue, neighborhood in zip(toronto_venues1['Venue Latitude'], toronto_venues1['Venue Longitude'],toronto_venues1['Venue'], toronto_venues1['Neighbourhood']):
    label = '{}, {}'.format(venue,neighborhood)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto)  
    
map_toronto

In [0]:
toronto_onehot = pd.get_dummies(toronto_venues1[['Venue Category']], prefix="", prefix_sep="")
toronto_onehot['Neighbourhood'] = toronto_venues1['Neighbourhood'] 
fixed_columns = [toronto_onehot.columns[-1]] + list(toronto_onehot.columns[:-1])
toronto_onehot = toronto_onehot[fixed_columns]
toronto_onehot.head()

,Neighbourhood,Adult Boutique,Aquarium,Art Gallery,Art Museum,Athletics & Sports,Bank,Bar,Baseball Stadium,Basketball Stadium,...,Strip Club,Supermarket,Swim School,Taco Place,Tanning Salon,Tea Room,Tennis Court,Theater,Trail,Wine Bar
4,"Harbourfront, Regent Park",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
7,"Harbourfront, Regent Park",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
8,"Harbourfront, Regent Park",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
9,"Harbourfront, Regent Park",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
19,"Harbourfront, Regent Park",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [0]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighbourhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighbourhood'] = toronto_grouped['Neighbourhood']

for ind in np.arange(toronto_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(toronto_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()  




,Neighbourhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,"Adelaide, King, Richmond",Hotel,Bar,Bookstore,Concert Hall,Gastropub,Wine Bar,Deli / Bodega,Opera House,Plaza,Office
1,Berczy Park,Cocktail Bar,Beer Bar,Pub,Creperie,Jazz Club,Hotel,Museum,Nightclub,Park,Fountain
2,"Brockton, Exhibition Place, Parkdale Village",Bar,Performing Arts Venue,Stadium,Wine Bar,General Travel,Dog Run,Event Space,Fountain,Gaming Cafe,Garden
3,Business Reply Mail Processing Centre 969 Eastern,Recording Studio,Garden,Park,Garden Center,Skate Park,Brewery,Wine Bar,Gay Bar,Dog Run,Event Space
4,"CN Tower, Bathurst Quay, Island airport, Harbo...",Boat or Ferry,Sculpture Garden,Harbor / Marina,General Travel,Dog Run,Event Space,Fountain,Gaming Cafe,Garden,Garden Center


In [0]:
toronto_grouped = toronto_onehot.groupby('Neighbourhood').mean().reset_index()
toronto_grouped.head()


,Neighbourhood,Adult Boutique,Aquarium,Art Gallery,Art Museum,Athletics & Sports,Bank,Bar,Baseball Stadium,Basketball Stadium,...,Strip Club,Supermarket,Swim School,Taco Place,Tanning Salon,Tea Room,Tennis Court,Theater,Trail,Wine Bar
0,"Adelaide, King, Richmond",0.0,0.0,0.0322581,0.0322581,0.0,0.0,0.0967742,0.0,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0322581,0.0,0.0322581
1,Berczy Park,0.0,0.0,0.0476190,0.0000000,0.0,0.0,0.0000000,0.0,0.047619,...,0.0,0.0,0.0,0.0,0.0,0.047619,0.0,0.0000000,0.0,0.0000000
2,"Brockton, Exhibition Place, Parkdale Village",0.0,0.0,0.0000000,0.0000000,0.0,0.0,0.3333333,0.0,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0000000,0.0,0.0000000
3,Business Reply Mail Processing Centre 969 Eastern,0.0,0.0,0.0000000,0.0000000,0.0,0.0,0.0000000,0.0,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0000000,0.0,0.0000000
4,"CN Tower, Bathurst Quay, Island airport, Harbo...",0.0,0.0,0.0000000,0.0000000,0.0,0.0,0.0000000,0.0,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0000000,0.0,0.0000000


In [0]:
kclusters = 7

toronto_grouped_clustering = toronto_grouped.drop('Neighbourhood', 1)
# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(toronto_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10] 

# add clustering labels
neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

toronto_merged = toronto_neigh

# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
toronto_merged = toronto_merged.join(neighborhoods_venues_sorted.set_index('Neighbourhood'), on='Neighbourhood')

toronto_merged.head()

,Postcode,Borough,Neighbourhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
2,M5A,Downtown Toronto,"Harbourfront, Regent Park",43.6542599,-79.3606359,2,Pub,Park,Theater,Spa,Event Space,Art Gallery,Hotel,Bank,Brewery,Performing Arts Venue
9,M5B,Downtown Toronto,"Ryerson, Garden District",43.6571618,-79.3789371,2,Plaza,Theater,Tea Room,Bar,Hookah Bar,Other Great Outdoors,Office,Music Venue,Movie Theater,Lounge
15,M5C,Downtown Toronto,St. James Town,43.6514939,-79.3754179,2,Hotel,Park,Gastropub,Cocktail Bar,Creperie,Beer Bar,Performing Arts Venue,Poke Place,Pub,Concert Hall
19,M4E,East Toronto,The Beaches,43.6763574,-79.2930312,3,Pub,Neighborhood,General Travel,Deli / Bodega,Dog Run,Event Space,Fountain,Gaming Cafe,Garden,Garden Center
20,M5E,Downtown Toronto,Berczy Park,43.6447708,-79.3733064,2,Cocktail Bar,Beer Bar,Pub,Creperie,Jazz Club,Hotel,Museum,Nightclub,Park,Fountain


In [0]:
# create map
map_clusters = folium.Map(location=[43.6532, -79.3832], zoom_start=12)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(toronto_merged['Latitude'], toronto_merged['Longitude'], toronto_merged['Neighbourhood'], toronto_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    cluster = int(cluster)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

In [0]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 0, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
24,Downtown Toronto,0,Bar,Spa,Wine Bar,Park,Gastropub,Office,Sculpture Garden,Poke Place,Tea Room,Art Museum
37,West Toronto,0,Bar,Cocktail Bar,Wine Bar,Salon / Barbershop,Art Gallery,Bistro,Boutique,Brewery,Dog Run,Music Venue
43,West Toronto,0,Bar,Performing Arts Venue,Stadium,Wine Bar,General Travel,Dog Run,Event Space,Fountain,Gaming Cafe,Garden
75,West Toronto,0,Dog Run,Movie Theater,Bank,Bar,Bookstore,Harbor / Marina,Event Space,Fountain,Gaming Cafe,Garden
80,Downtown Toronto,0,Bookstore,Bar,Pub,Nightclub,Beer Bar,College Arts Building,Noodle House,Poutine Place,Jazz Club,Theater
84,Downtown Toronto,0,Bar,Gaming Cafe,Wine Bar,Bistro,Park,Deli / Bodega,Hotel Bar,Organic Grocery,Poutine Place,Cocktail Bar


In [0]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 1, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
25,Downtown Toronto,1,Park,Athletics & Sports,Nightclub,Wine Bar,Harbor / Marina,Event Space,Fountain,Gaming Cafe,Garden,Garden Center
47,East Toronto,1,Park,Pub,Movie Theater,Brewery,Wine Bar,General Entertainment,Dog Run,Event Space,Fountain,Gaming Cafe
61,Central Toronto,1,Photography Studio,Swim School,Park,Wine Bar,General Entertainment,Dog Run,Event Space,Fountain,Gaming Cafe,Garden
74,Central Toronto,1,Park,Pub,Pharmacy,History Museum,Gay Bar,Deli / Bodega,Dog Run,Event Space,Fountain,Gaming Cafe
91,Downtown Toronto,1,Park,Trail,Playground,Irish Pub,Dog Run,Event Space,Fountain,Gaming Cafe,Garden,Garden Center


In [0]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 2, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
2,Downtown Toronto,2,Pub,Park,Theater,Spa,Event Space,Art Gallery,Hotel,Bank,Brewery,Performing Arts Venue
9,Downtown Toronto,2,Plaza,Theater,Tea Room,Bar,Hookah Bar,Other Great Outdoors,Office,Music Venue,Movie Theater,Lounge
15,Downtown Toronto,2,Hotel,Park,Gastropub,Cocktail Bar,Creperie,Beer Bar,Performing Arts Venue,Poke Place,Pub,Concert Hall
20,Downtown Toronto,2,Cocktail Bar,Beer Bar,Pub,Creperie,Jazz Club,Hotel,Museum,Nightclub,Park,Fountain
30,Downtown Toronto,2,Hotel,Bar,Bookstore,Concert Hall,Gastropub,Wine Bar,Deli / Bodega,Opera House,Plaza,Office
31,West Toronto,2,Pharmacy,Supermarket,Pool,Brewery,Bank,Bar,Park,Music Venue,Dog Run,Event Space
36,Downtown Toronto,2,Hotel,Aquarium,Scenic Lookout,Brewery,Music Venue,Park,Baseball Stadium,Sports Bar,History Museum,Bar
41,East Toronto,2,Bookstore,Juice Bar,Spa,Pub,Brewery,Trail,Bank,Harbor / Marina,Event Space,Fountain
42,Downtown Toronto,2,Hotel,Gastropub,Deli / Bodega,Bar,Beer Bar,Wine Bar,Basketball Stadium,Lounge,General Travel,Museum
48,Downtown Toronto,2,Hotel,Gastropub,Deli / Bodega,Bar,Cocktail Bar,Museum,General Travel,Monument / Landmark,Jazz Club,Building


In [0]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 3, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
19,East Toronto,3,Pub,Neighborhood,General Travel,Deli / Bodega,Dog Run,Event Space,Fountain,Gaming Cafe,Garden,Garden Center
86,Central Toronto,3,Pub,Supermarket,Sports Bar,Wine Bar,General Entertainment,Deli / Bodega,Dog Run,Event Space,Fountain,Gaming Cafe


In [0]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 4, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
87,Downtown Toronto,4,Boat or Ferry,Sculpture Garden,Harbor / Marina,General Travel,Dog Run,Event Space,Fountain,Gaming Cafe,Garden,Garden Center


In [0]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 5, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
83,Central Toronto,5,Trail,Tennis Court,Playground,General Travel,Dog Run,Event Space,Fountain,Gaming Cafe,Garden,Garden Center


In [0]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 6, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
68,Central Toronto,6,Trail,Dance Studio,Deli / Bodega,Dog Run,Event Space,Fountain,Gaming Cafe,Garden,Garden Center,Gastropub
